In [1]:
import pandas as pd
import pickle

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error

In [3]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/ulya/Desktop/mlops_zoomcamp/mlops-zoomcamp/03-orchestration/artifacts/1', creation_time=1750058530984, experiment_id='1', last_update_time=1750058530984, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    return df

In [6]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [7]:
categorical = ['PU_DO'] 
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
import xgboost as xgb

In [12]:
with mlflow.start_run():
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learing_rate': 0.09585355369315604,
        'max_depth': 30, 
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_lambda': 0.18060244040060163,
        'reg_alpha': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params = best_params,
        dtrain = train,
        num_boost_round=30,
        evals=[(valid, "validation")],
        early_stopping_rounds=2
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open ("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
    


/home/ulya/anaconda3/envs/mlops_zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:08:13] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/ulya/anaconda3/envs/mlops_zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:08:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "learing_rate" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:7.96299
[1]	validation-rmse:6.66984
[2]	validation-rmse:5.91042
[3]	validation-rmse:5.47918
[4]	validation-rmse:5.23935
[5]	validation-rmse:5.10398
[6]	validation-rmse:5.02430
[7]	validation-rmse:4.97565
[8]	validation-rmse:4.94415
[9]	validation-rmse:4.92553
[10]	validation-rmse:4.91145
[11]	validation-rmse:4.90138
[12]	validation-rmse:4.89364
[13]	validation-rmse:4.88546
[14]	validation-rmse:4.88021
[15]	validation-rmse:4.87511
[16]	validation-rmse:4.87267
[17]	validation-rmse:4.87056
[18]	validation-rmse:4.86818
[19]	validation-rmse:4.86599
[20]	validation-rmse:4.86381
[21]	validation-rmse:4.86211
[22]	validation-rmse:4.86015
[23]	validation-rmse:4.85809
[24]	validation-rmse:4.85628
[25]	validation-rmse:4.85458
[26]	validation-rmse:4.85264
[27]	validation-rmse:4.85084
[28]	validation-rmse:4.84900
[29]	validation-rmse:4.84728


/home/ulya/anaconda3/envs/mlops_zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:09:08] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/06/16 11:09:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run spiffy-dog-951 at: http://localhost:5000/#/experiments/1/runs/009a839035e942adb9cee4d555b5f85c
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [ ]:
logged_model = 'runs:/5142c0365b77438c83a364ce549d3c60/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
xgboost_model = mlflow.xgboost.load_model(loaded_model)